# Layer Norm

**Descrição**  
Layer Normalization (LayerNorm) é uma técnica de normalização que ajusta as ativações de uma rede para ficarem mais estáveis durante o treinamento. Em vez de depender do tamanho do batch (como no BatchNorm), o LayerNorm normaliza **por amostra** (e, em Transformers, tipicamente **por token**), ao longo da dimensão do embedding.

**Objetivo**  
- Aumentar a **estabilidade** do treinamento (reduzindo problemas de gradiente explodindo/sumindo).  
- Ajudar redes profundas (ex.: **Transformers**) a aprenderem com mais consistência.  
- Manter as ativações em uma escala mais controlada, facilitando a otimização.

**Funcionamento**  
Dado um tensor `x` com shape `(..., emb_dim)`, o LayerNorm normaliza os valores ao longo da última dimensão (`emb_dim`):

1. Calcula a média `μ` e variância `σ²` ao longo de `emb_dim`  
2. Normaliza:
  <img src="https://latex.codecogs.com/svg.image?\text{norm}(x)=\frac{x-\mu}{\sqrt{\sigma^2&plus;\epsilon}}" />
2. Aplica uma transformação afim aprendível:
<img src="https://latex.codecogs.com/svg.image?y=\gamma\cdot\text{norm}(x)&plus;\beta&space;" />

Onde:
- `γ` (`scale`) e `β` (`shift`) são parâmetros treináveis com shape `(emb_dim,)`
- `ε` é um valor pequeno para evitar divisão por zero

Em um tensor `(batch, seq_len, emb_dim)`, isso significa que **cada token** (cada posição em `seq_len`) é normalizado de forma independente, usando apenas seus próprios valores do embedding.


In [1]:
import torch
from torch import nn

## Pra que serve o `LayerNorm` (Layer Normalization)?

O **LayerNorm** é uma técnica de normalização usada para **estabilizar e acelerar o treinamento** de redes profundas (muito comum em **Transformers/LLMs**).

### O que ele faz?

Para cada vetor de embedding (por exemplo, **cada token** em uma sequência), ele:

1. **Calcula a média** dos valores na dimensão do embedding  
2. **Calcula a variância** nessa mesma dimensão  
3. **Normaliza** os valores para ficarem com média ~0 e variância ~1  
4. Aplica uma transformação aprendível:
   - `scale` (γ): ajusta a escala final
   - `shift` (β): ajusta o deslocamento final

A ideia é:

<img src="https://latex.codecogs.com/svg.image?\[\text{norm}(x)=\frac{x-\mu}{\sqrt{\sigma^2&plus;\epsilon}},\quad&space;y=\gamma\cdot\text{norm}(x)&plus;\beta\]" />


### Por que isso ajuda?

- **Deixa o treino mais estável**: evita ativações com escalas muito diferentes entre camadas.
- **Melhora o fluxo de gradiente**: ajuda a treinar arquiteturas profundas com menos problemas de gradiente explodindo/sumindo.
- **Não depende do batch**: diferente de BatchNorm, o LayerNorm normaliza **por amostra/token**, então funciona bem com batch pequeno e com sequências variáveis.

### Qual dimensão é normalizada no seu código?

No seu `forward`:

```python
mean = x.mean(dim=-1, keepdim=True)
var = x.var(dim=-1, keepdim=True, unbiased=False)
````

Isso normaliza **ao longo da última dimensão** (`dim=-1`), que normalmente é `emb_dim`.

Se `x` tem shape `(batch, seq_len, emb_dim)`, então:

* para cada **exemplo do batch**
* para cada **posição (token) em `seq_len`**
* normaliza os **valores do embedding** daquele token

### Resumo

**LayerNorm serve para manter as ativações em uma escala mais controlada por token, deixando o treinamento de Transformers mais estável e eficiente.**

In [2]:
class LayerNorm(nn.Module):
    """
    Implementação de Layer Normalization (normalização por token, ao longo da última dimensão).

    Esta camada normaliza as ativações ao longo da dimensão de embedding (dim=-1),
    e então aplica uma transformação afim aprendível:

        y = scale * (x - mean) / sqrt(var + eps) + shift

    Parâmetros:
    ----------
    emb_dim : int
        Dimensão do embedding (tamanho do último eixo de `x`).
    eps : float, default = 1e-5
        Constante pequena para evitar divisão por zero.

    Entrada:
    -------
    x : torch.Tensor
        Tensor com shape (..., emb_dim). Exemplos comuns:
        - (batch, seq_len, emb_dim)
        - (seq_len, emb_dim)

    Saída:
    -----
    torch.Tensor
        Tensor normalizado com o mesmo shape de `x`.

    Observações:
    -----------
    - `unbiased=False` em `var` corresponde ao comportamento típico em LN.
    - `scale` e `shift` têm shape (emb_dim,) e são broadcastados para o shape de `x`.

    Referência:
    ----------
    Seção 4.2 — "Normalizing activations with layer normalization". :contentReference[oaicite:0]{index=0}
    """

    def __init__(self, emb_dim: int, eps: float = 1e-5) -> None:
        super().__init__()

        if not isinstance(emb_dim, int) or emb_dim <= 0:
            raise ValueError("emb_dim deve ser um int positivo.")

        self.eps = float(eps)
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Executa a normalização por camada (LayerNorm) ao longo do último eixo.

        Parâmetros:
        ----------
        x : torch.Tensor
            Tensor de entrada com shape (..., emb_dim).

        Retorno:
        -------
        torch.Tensor
            Tensor normalizado com o mesmo shape de `x`.
        """
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        norm_x = (x - mean) / torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift

In [3]:
def media_desvio_por_linha(x: torch.Tensor) -> tuple[torch.Tensor, torch.Tensor]:
    """
    Calcula média e desvio padrão ao longo da última dimensão (dim=-1),
    retornando 1 valor por linha (ou por token, se for o caso).
    """
    mean = x.mean(dim=-1)
    std = x.std(dim=-1, unbiased=False)  # condizente com var(..., unbiased=False)
    return mean, std


# ------------------------------------------------------------
# Exemplo de uso
# ------------------------------------------------------------
torch.manual_seed(123)

batch_example = torch.randn(2, 5)

mean_before, std_before = media_desvio_por_linha(batch_example)

ln = LayerNorm(emb_dim=5)
out = ln(batch_example)

mean_after, std_after = media_desvio_por_linha(out)

print("Entrada (batch_example):")
print(batch_example)
print("\nMédia por linha (antes):", mean_before)
print("Desvio padrão por linha (antes):", std_before)

print("\nSaída após LayerNorm:")
print(out)
print("\nMédia por linha (depois):", mean_after)
print("Desvio padrão por linha (depois):", std_after)

print("\nChecagem (aprox.):")
# erro absoluto máximo da média e do std em relação a (0 e 1)
max_mean_err = (mean_after).abs().max().item()
max_std_err = (std_after - 1).abs().max().item()

print("Max |mean - 0|:", max_mean_err)
print("Max |std  - 1|:", max_std_err)

print(
    "Média ~ 0 ?", torch.allclose(mean_after, torch.zeros_like(mean_after), atol=1e-6)
)
print(
    "Std ~ 1   ?", torch.allclose(std_after, torch.ones_like(std_after), atol=1e-4)
)  # atol mais realista

Entrada (batch_example):
tensor([[-0.1115,  0.1204, -0.3696, -0.2404, -1.1969],
        [ 0.2093, -0.9724, -0.7550,  0.3239, -0.1085]])

Média por linha (antes): tensor([-0.3596, -0.2606])
Desvio padrão por linha (antes): tensor([0.4489, 0.5170])

Saída após LayerNorm:
tensor([[ 0.5528,  1.0693, -0.0223,  0.2656, -1.8654],
        [ 0.9087, -1.3767, -0.9564,  1.1304,  0.2940]], grad_fn=<AddBackward0>)

Média por linha (depois): tensor([-2.9802e-08,  0.0000e+00], grad_fn=<MeanBackward1>)
Desvio padrão por linha (depois): tensor([1.0000, 1.0000], grad_fn=<StdBackward0>)

Checagem (aprox.):
Max |mean - 0|: 2.9802322387695312e-08
Max |std  - 1|: 2.47955322265625e-05
Média ~ 0 ? True
Std ~ 1   ? True
